## Initialisation

### Imports

In [1]:
from langchain_community.document_loaders.text import TextLoader
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import numpy as np
import pickle

### Basic functions and parameters

In [2]:
def pretty_print_docs(docs):
    if isinstance(docs[0], tuple):
        print(
            f"\n{'-' * 100}\n".join(
                [
                    f"Document {i+1}: ({d[1]})\n\n{d[0].page_content}\nMetadata: {d[0].metadata}"
                    for i, d in enumerate(docs)
                ]
            )
        )
    else:
        print(
            f"\n{'-' * 100}\n".join(
                [
                    f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                    for i, d in enumerate(docs)
                ]
            )
        )

def score_normalizer(val: float) -> float:
    return 1 - (1 / (1 + np.exp(val)))

# Define your custom similarity calculation function
def custom_relevance_score_fn(similarity_score: float) -> float:
    # Example calculation (customize as needed)
    relevance_score = 1 / (1 + np.exp(-similarity_score))
    return relevance_score

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

### Document split and embedding

In [3]:
document_names = "nous_sommes_en_guerre.txt", "jeux_olympiques_paris.txt", "pacte_novation_linkedin.txt"

loaders = [TextLoader(document_name, encoding="utf-8") for document_name in document_names]

documents = []
for loader in loaders:
    documents.extend(loader.load())

embedding = OllamaEmbeddings(model="mistral")


## Vectorstore

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

docs = text_splitter.split_documents(documents)

for idx, text in enumerate(docs):
    text.metadata["id"] = idx

### FAISS

In [5]:
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy

In [6]:
vectordb = None
vectordb = FAISS.from_documents(
    docs, 
    embedding, 
    normalize_L2=True, 
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE, 
    relevance_score_fn=custom_relevance_score_fn
    )
vectordb.save_local("local_vectorstore_faiss")

In [ ]:
vectordb = None
vectordb = FAISS.load_local(
    "local_vectorstore_faiss", 
    embeddings=embedding, 
    allow_dangerous_deserialization=True, 
    normalize_L2=True, 
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE, 
    relevance_score_fn=custom_relevance_score_fn
    )

In [7]:
query = """président Thomas Bach"""
docs_proba = vectordb.similarity_search_with_score(query, k=3)
pretty_print_docs(docs_proba)

Document 1: (0.8933698534965515)

Le 7 décembre 2020, le programme est arrêté par la commission exécutive du CIO. Quatre sports additionnels sont confirmés : le surf, le skateboard, l'escalade et le breakdance (les trois premiers étant déjà au programme des Jeux de Tokyo précédents). De plus, plusieurs modifications d'épreuves sont effectuées, entraînant d'une part une réduction du quota d'athlètes nécessaire pour réduire la complexité d'organisation dans le contexte pandémique, et d'autre part une parité parfaite de ces Jeux avec autant d'hommes que de femmes engagés :
Metadata: {'source': 'jeux_olympiques_paris.txt', 'id': 35}
----------------------------------------------------------------------------------------------------
Document 2: (0.9175640940666199)

•	Assigraph International,  est depuis 1978 un des éditeurs précurseurs de la CAO dédiée aux métiers de la schématique. Grâce à son équipe d’experts métiers et son partenariat avec un des leaders mondiaux de la CAO, Autodesk, le

In [8]:
query = """président Thomas Bach"""
docs_proba = vectordb.similarity_search_with_relevance_scores(query, k=3)
pretty_print_docs(docs_proba)

Document 1: (0.7095850930428672)

Le 7 décembre 2020, le programme est arrêté par la commission exécutive du CIO. Quatre sports additionnels sont confirmés : le surf, le skateboard, l'escalade et le breakdance (les trois premiers étant déjà au programme des Jeux de Tokyo précédents). De plus, plusieurs modifications d'épreuves sont effectuées, entraînant d'une part une réduction du quota d'athlètes nécessaire pour réduire la complexité d'organisation dans le contexte pandémique, et d'autre part une parité parfaite de ces Jeux avec autant d'hommes que de femmes engagés :
Metadata: {'source': 'jeux_olympiques_paris.txt', 'id': 35}
----------------------------------------------------------------------------------------------------
Document 2: (0.7145455268714127)

•	Assigraph International,  est depuis 1978 un des éditeurs précurseurs de la CAO dédiée aux métiers de la schématique. Grâce à son équipe d’experts métiers et son partenariat avec un des leaders mondiaux de la CAO, Autodesk, le

### Chroma

In [9]:
from langchain.vectorstores.chroma import Chroma

In [10]:
vectordb = None
vectordb = Chroma.from_documents(
    docs,
    embedding,
    persist_directory="local_vectorstore_chroma",
    relevance_score_fn=custom_relevance_score_fn,
    )

In [ ]:
vectordb = None
vectordb = Chroma(
    embedding_function=embedding,
    persist_directory="local_vectorstore_chroma",
    relevance_score_fn=custom_relevance_score_fn,
    )

In [11]:
query = """c'est quoi la meilleure discipline olympique ?"""
docs_proba = vectordb.similarity_search_with_score(query, k=4)
pretty_print_docs(docs_proba)

Document 1: (130151.0391394958)

Le gouvernement, dès demain, précisera toutes ces mesures. Elles seront en fonction des besoins, des réalités économiques, des nécessités secteur par secteur, évidemment adaptées. Nous serons au rendez-vous pour que notre économie soit préservée dans cette période si dure et pour que l’ensemble des travailleuses et des travailleurs puissent avoir cette sécurité aussi en termes de pouvoir d’achat, de continuité de leur vie.
Metadata: {'id': 19, 'source': 'nous_sommes_en_guerre.txt'}
----------------------------------------------------------------------------------------------------
Document 2: (130151.0391394958)

Le gouvernement, dès demain, précisera toutes ces mesures. Elles seront en fonction des besoins, des réalités économiques, des nécessités secteur par secteur, évidemment adaptées. Nous serons au rendez-vous pour que notre économie soit préservée dans cette période si dure et pour que l’ensemble des travailleuses et des travailleurs puissent avo

In [12]:
query = """c'est quoi la meilleure discipline olympique ?"""
docs_proba = vectordb.similarity_search_with_relevance_scores(query, k=30)
pretty_print_docs(docs_proba)

Document 1: (1.0)

Le gouvernement, dès demain, précisera toutes ces mesures. Elles seront en fonction des besoins, des réalités économiques, des nécessités secteur par secteur, évidemment adaptées. Nous serons au rendez-vous pour que notre économie soit préservée dans cette période si dure et pour que l’ensemble des travailleuses et des travailleurs puissent avoir cette sécurité aussi en termes de pouvoir d’achat, de continuité de leur vie.
Metadata: {'id': 19, 'source': 'nous_sommes_en_guerre.txt'}
----------------------------------------------------------------------------------------------------
Document 2: (1.0)

Le gouvernement, dès demain, précisera toutes ces mesures. Elles seront en fonction des besoins, des réalités économiques, des nécessités secteur par secteur, évidemment adaptées. Nous serons au rendez-vous pour que notre économie soit préservée dans cette période si dure et pour que l’ensemble des travailleuses et des travailleurs puissent avoir cette sécurité aussi en t

## Multi Vector 

In [ ]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.retrievers.multi_vector import SearchType
from langchain.vectorstores.chroma import Chroma
from langchain.storage import InMemoryByteStore
import uuid

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)

In [ ]:
vectordb = None
vectordb = Chroma(collection_name="full_documents", embedding_function=embedding, persist_directory="local_multivector_chroma")

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# the retriever
retriever = MultiVectorRetriever(
    vectorstore=vectordb,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [ ]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [ ]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [ ]:
retriever.vectorstore = None
retriever.vectorstore.add_documents(sub_docs, persist_directory="local_multivector_chroma")
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
# load the retriever vectorstore
retriever.vectorstore = None
retriever.vectorstore = Chroma(collection_name="full_documents", embedding_function=embedding, persist_directory="local_multivector_chroma")
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
query = """c'est quoi la meilleure discipline olympique ?"""
docs_proba = retriever.vectorstore.similarity_search_with_score(query, k=4)
pretty_print_docs(docs_proba)

In [ ]:
# Retriever returns larger chunks
retriever.search_type = SearchType.mmr
print("\n######\n".join(
    [d.page_content 
     for d in retriever.get_relevant_documents(
         """c'est quoi la meilleure discipline olympique ?"""
         )
    ]
))